<a href="https://colab.research.google.com/github/Medissaoui07/Fine-Tuning-MobileNetV2-for-Classification-/blob/main/fine_tuning__MobileNetV2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
kagglehub.login()


In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

sid4sal_alpaca_dataset_small_path = kagglehub.dataset_download('sid4sal/alpaca-dataset-small')
mohamedissaouiii_testing_path = kagglehub.dataset_download('mohamedissaouiii/testing')
mohamedissaouiii_testing1_path = kagglehub.dataset_download('mohamedissaouiii/testing1')
mohamedissaouiii_testing2_path = kagglehub.dataset_download('mohamedissaouiii/testing2')
mohamedissaouiii_testing3_path = kagglehub.dataset_download('mohamedissaouiii/testing3')
mohamedissaouiii_humann_path = kagglehub.dataset_download('mohamedissaouiii/humann')

print('Data source import complete.')


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os
import tensorflow as tf
import tensorflow.keras.layers as tfl

from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.layers import RandomFlip, RandomRotation

In [ ]:
directory="/kaggle/input/alpaca-dataset-small/dataset"
img_size=(160,160)

train_data=image_dataset_from_directory(directory,shuffle=True,batch_size=32,image_size=img_size , validation_split=0.2 , subset='training',seed=123)
val_data=image_dataset_from_directory(directory,shuffle=True,batch_size=32,image_size=img_size , validation_split=0.2 , subset='validation',seed=123)

In [ ]:
class_names=train_data.class_names

plt.figure(figsize=(10,10))
for img , labels in train_data.take(1):
    for i in range(9):
        ax=plt.subplot(3,3,i+1)
        plt.imshow(img[i].numpy().astype('uint8'))
        plt.title(class_names[labels[i]])


In [ ]:
AUTOTUNE = tf.data.experimental.AUTOTUNE
train_data = train_data.prefetch(buffer_size=AUTOTUNE)

In [ ]:
def data_augmenter():
  data_aug=tf.keras.Sequential()
  data_aug.add(RandomFlip('horizontal'))
  data_aug.add(RandomRotation(0.2))
  return data_aug

In [ ]:
data_augmentation = data_augmenter()

for image, _ in train_data.take(1):
    plt.figure(figsize=(10, 10))
    first_image = image[0]
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        augmented_image = data_augmentation(tf.expand_dims(first_image, 0))
        plt.imshow(augmented_image[0] / 255)
        plt.axis('off')

In [ ]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context
base_model = tf.keras.applications.MobileNetV2(input_shape=img_size+(3,),
                                               include_top=True,
                                               weights='imagenet')


In [ ]:
base_model.summary()


In [ ]:
import tensorflow.keras.layers as tfl

preprocess_input=tf.keras.applications.mobilenet_v2.preprocess_input

def alpaca_model(img_size , data_augmentation=data_augmenter()):
    input_shape=img_size+(3,)

    base_model=tf.keras.applications.MobileNetV2(input_shape,
                                               include_top=False,
                                               weights='imagenet')
    base_model.trainable=False

    input=tf.keras.Input(input_shape)

    x=data_augmentation(input)

    x=preprocess_input(x)

    x=base_model(x,training=False)

    x=tfl.GlobalAveragePooling2D()(x)

    x=tfl.Dropout(0.3)(x)


    prediction_layer=tfl.Dense(1)

    output=prediction_layer(x)

    model=tf.keras.Model(input , output)
    return model





In [ ]:
model=alpaca_model(img_size)

In [ ]:
optimizer=tf.keras.optimizers.Adam(learning_rate=0.01)
loss=tf.keras.losses.BinaryCrossentropy(from_logits=True)
model.compile(optimizer=optimizer ,
             loss=loss,
             metrics=['accuracy'])

In [ ]:
history = model.fit(train_data, validation_data=val_data, epochs=10)


In [ ]:
class_names

In [ ]:
base_model.trainable=True

print(len(base_model.layers))

In [ ]:
fine_tune_at=126
lr=0.0001

for layer in base_model.layers[:fine_tune_at]:
    layer.trainable=False
loss_function=tf.keras.losses.BinaryCrossentropy(from_logits=True)
optimizer=tf.keras.optimizers.Adam(learning_rate=lr)
model.compile(loss=loss_function , optimizer=optimizer , metrics=['accuracy'])


In [ ]:
training_epochs=20
model.fit(train_data ,
         epochs=training_epochs ,
         initial_epoch=history.epoch[-1] ,
         validation_data=val_data)